In [1]:
import tensorflow as tf
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
import matplotlib.patches as patches
import cv2
import os
from tqdm import tqdm
import maybe_download

/home/vipin/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
maybe_download.maybe_download()

Files already downloaded
Files already extracted


In [3]:
imsize = (416, 416)
pb_dir = './ssd_inception_v2_coco_2017_11_17/frozen_inference_graph.pb'
log_dir = './tmp/'
img_dir = './Test-Images/'
vid_out_path = './Test-Images/Test-Video/Results'
img_path = []
for im_p in os.listdir(img_dir):
    if 'jpg' in im_p:
        img_path.append(img_dir+im_p)
image_fed = []
for i in range(len(img_path)):
    im_tmp = np.array(Image.open(img_path[i]), dtype=np.uint8)
    image_fed.append(im_tmp)
image_fed = np.asarray(image_fed)
print(image_fed.shape)

(16, 416, 416, 3)


In [4]:
graph = tf.Graph()

In [5]:
with graph.as_default():
    with tf.gfile.FastGFile(pb_dir, 'rb') as file:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(file.read())
        tf.import_graph_def(graph_def, name='')
        
        img = graph.get_tensor_by_name('image_tensor:0')
        detection_boxes = graph.get_tensor_by_name('detection_boxes:0')
        detection_scores = graph.get_tensor_by_name('detection_scores:0')
        num_detections = graph.get_tensor_by_name('num_detections:0')
        detection_classes = graph.get_tensor_by_name('detection_classes:0')
        print(img, detection_boxes, detection_scores, num_detections, sep='\n')
        sess = tf.Session(graph=graph)
        file_writer = tf.summary.FileWriter(log_dir, graph=graph)

Tensor("image_tensor:0", shape=(?, ?, ?, 3), dtype=uint8)
Tensor("detection_boxes:0", shape=(?, 100, 4), dtype=float32)
Tensor("detection_scores:0", shape=(?, 100), dtype=float32)
Tensor("num_detections:0", shape=(?,), dtype=float32)


In [14]:
vid_inp = './Test-Images/Test-Video/traffic_sample_1.mp4'
vid_out = './Test-Images/Test-Video/Results/traffic_sample_1_bbox.mp4'
if not os.path.exists(vid_out):
    os.mkdir(vid_out_path)
num_iter = 67
batch_size = 32
num_pred = 30
video_reader = cv2.VideoCapture(vid_inp)
video_writer = cv2.VideoWriter(vid_out, cv2.VideoWriter_fourcc(*'XVID'), 30.0, imsize)
nb_frames = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
frame_h = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_w = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))

print(nb_frames, frame_h, frame_w)


def draw_boxes(image_fed, bbox):
    
    for i in range(best_boxes_roi.shape[0]):
        im = np.reshape(image_fed[i], (416, 416, 3))
#         fix, ax = plt.subplots()
#         ax.imshow(im)
#         print(best_boxes_classes[i])
        for j in range(num_pred):
            if best_boxes_scores[i][j] > 0.15:
                x = best_boxes_roi[i][j][1]
                y = best_boxes_roi[i][j][0]
                x_max = best_boxes_roi[i][j][3]
                y_max = best_boxes_roi[i][j][2]
#                 rect = patches.Rectangle((x, y), (x_max - x), (y_max - y), linewidth=2, edgecolor='r', facecolor='none')
#                 ax.add_patch(rect)
                cv2.rectangle(im, (x,y), (x_max,y_max), (0,255,0), 3)
#         plt.show()
        video_writer.write(im)


for i in tqdm(range(num_iter)):
    image_bat = []
    for j in range(batch_size):
        ret, image = video_reader.read()
        image = cv2.resize(image, imsize)
        image_bat.append(image)
        image_batch = np.asarray(image_bat)
    feed_dict = {img:image_batch}
    y_p_boxes, y_p_scores, y_p_num_detections, y_p_classes = sess.run([detection_boxes, 
                                                                       detection_scores, 
                                                                       num_detections, 
                                                                       detection_classes], feed_dict=feed_dict)
    best_boxes_roi = []
    best_boxes_scores = []
    best_boxes_classes = []
    for i in range(y_p_boxes.shape[0]):
        temp = y_p_boxes[i, :num_pred] * imsize[0]
        best_boxes_roi.append(temp)
        best_boxes_scores.append(y_p_scores[i, :num_pred])
        best_boxes_classes.append(y_p_classes[i, :num_pred])
    best_boxes_roi = np.asarray(best_boxes_roi)
    best_boxes_scores = np.asarray(best_boxes_scores)
    best_boxes_classes = np.asarray(best_boxes_classes)
    draw_boxes(image_batch, best_boxes_roi)
    
print(y_p_boxes.shape, y_p_scores.shape, y_p_num_detections.shape, y_p_classes.shape)
        

  0%|          | 0/67 [00:00<?, ?it/s]

2175 360 480


100%|██████████| 67/67 [01:42<00:00,  1.53s/it]

(32, 100, 4) (32, 100) (32,) (32, 100)
